In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 838.1 kB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
!pip install requests==2.32.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.


In [5]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.1 MB/s eta 0:00:00


In [3]:

import requests
import langchain
print("Requests:", requests.__version__)
print("LangChain:", langchain.__version__)


Requests: 2.32.4
LangChain: 1.2.3


# **Built in Tools**

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who is the current president of the United States?")
print(results)

The president of the United States ( POTUS ) b is the head of state and head of government of the United States . ... the later office of president of ... The President of the United States (often abbreviated "POTUS") is the head of ... The 43rd and current President of the United States is George W. Donald Trump is the 47th and current president of the United States , in office since January 20, 2025. ... The president -elect of the United States is ... G2G: Who is the current President of the United States ? ... Who would have thought that the President of the United States is my cousin? The current President is Donald Trump , who won a ... Condemning slavery is the baseline for anyone who wants to be president of the United States .


# **Custom Tools**

## Using @tool decorator

In [2]:
from langchain_core.tools import tool

In [3]:
## Step 1 : create a function

def multiply(a,b):
  """Multiply two numbers"""
  return a*b

In [4]:
## Step 2 : add type hints

def multiply(a:int , b:int) -> int :
  """Multiply two numbers"""
  return a*b

In [5]:
## Step 3 : Add tool decorator

@tool
def multiply(a:int,b:int)->int:
  """Multiply two numbers"""
  return a*b

In [6]:
result = multiply.invoke({'a':3,'b':4})
result

12

## Using StructuredTool

In [11]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel,Field

In [12]:
class MultiplyInput(BaseModel):
  a : int = Field(required = True , description = 'The first number to multiply')
  b : int = Field(required = True , description = 'The second number to multiply')

/tmp/ipython-input-3049411355.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a : int = Field(required = True , description = 'The first number to multiply')
/tmp/ipython-input-3049411355.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b : int = Field(required = True , description = 'The second number to multiply')


In [13]:
def multiply_func(a,b):
  return a*b

In [14]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = 'Multiply',
    description = 'Multiply two numbers',
    args_schema = MultiplyInput
)

In [17]:
result = multiply_tool.invoke({'a':3,'b':4})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

12
Multiply
Multiply two numbers
{'a': {'description': 'The first number to multiply', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'required': True, 'title': 'B', 'type': 'integer'}}


## Using BaseTool

In [18]:
from langchain_core.tools import BaseTool
from typing import Type

In [19]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
  a : int = Field(required = True , description = 'The first number to multiply')
  b : int = Field(required = True , description = 'The second number to multiply')

/tmp/ipython-input-2694921232.py:4: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a : int = Field(required = True , description = 'The first number to multiply')
/tmp/ipython-input-2694921232.py:5: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b : int = Field(required = True , description = 'The second number to multiply')


In [20]:
class MultiplyTool(BaseTool):
  name : str = 'multiply'
  description : str = 'Multiply two number'
  arg_schema : Type[BaseModel] = MultiplyInput

  def _run(self, a : int , b : int) -> int :
    return a*b

In [22]:
result = MultiplyTool().invoke({'a':3,'b':4})
print(result)
print(MultiplyTool().name)
print(MultiplyTool().description)
print(MultiplyTool().arg_schema)

12
multiply
Multiply two number
<class '__main__.MultiplyInput'>


## Toolkit

In [23]:
from langchain_core.tools import tool

# custom tools

@tool
def add(a:int,b:int) -> int :
  """Add two numbers"""
  return a+b

@tool
def multiply(a:int,b:int) -> int :
  """Multiply two numbers"""
  return a*b

In [24]:
class MathToolkit :
  def get_tools(self):
    return [add,multiply]

In [25]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools :
  print(tool.name,"=>",tool.description)

add => Add two numbers
multiply => Multiply two numbers


In [29]:
tools[0].invoke({"a": 2, "b": 3})

5

In [30]:
tools[1].invoke({"a": 3, "b": 4})

12